In [1]:
from typing import Dict, Any
import asyncio
from gpt_researcher import GPTResearcher

In [3]:
from langchain_aws import BedrockEmbeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

In [4]:
class CustomLogsHandler:
    """A custom Logs handler class to handle JSON data."""
    def __init__(self):
        self.logs = []  # Initialize logs to store data

    async def send_json(self, data: Dict[str, Any]) -> None:
        """Send JSON data and log it."""
        self.logs.append(data)  # Append data to logs
        # print(f"My custom Log: {data}")  # For demonstration, print the log

In [27]:
query = "why is Nvidia stock going up?"
report_type = "research_report"  # Type of report to generate
report_source = "online"  # Could specify source like 'online', 'books', etc.
tone = "informative"  # Tone of the report ('informative', 'casual', etc.)
config_path = None  # Path to a config file, if needed

# Initialize researcher with a custom WebSocket
custom_logs_handler = CustomLogsHandler()

# researcher = GPTResearcher(
#     query=query,
#     report_type=report_type,
#     report_source=report_source,
#     tone=tone,
#     config_path=config_path,
#     websocket=custom_logs_handler
# )


client = QdrantClient(
    url="https://ft-vdb.epiphaiplatform.com:443",
    api_key="Y3NMMGK3Okzt7rzho88jhmzPZl5Mhhnd98i39bLG4OJPGRtNV7pH8sOlVNtveGce",
)
vector_store = QdrantVectorStore(
    client=client,
    content_payload_key="content",
    # metadata_payload_key="document_key",
    collection_name="epiphai_collection",
    embedding=BedrockEmbeddings(region_name="us-east-1", model_id="amazon.titan-embed-text-v2:0"),
)

In [ ]:
from qdrant_client import models

file_filters = "<file1>,<file2>".split(",")

vector_store_filters = models.Filter(
    [
        models.FieldCondition(
            key="document_key",
            match=models.MatchText(text=file)
        ),
    ] for file in file_filters
)

In [ ]:
researcher = GPTResearcher(
    query,
    report_type,
    "markdown",
    "langchain_vectorstore",
    tone,
    vector_store=vector_store,
    fvector_store_filters=vector_store_filters
    config_path=config_path,
    websocket=custom_logs_handler,
    verbose=True
)

research = await researcher.conduct_research()  # Conduct the research

INFO:     [19:15:25] 🔍 Starting the research task for 'why is Nvidia stock going up?'...
INFO:     [19:15:25] 💰 Finance Agent
INFO:     [19:15:25] 🌐 Browsing the web to learn more about the task: why is Nvidia stock going up?...
INFO:     [19:15:26] 🤔 Planning the research strategy and subtasks...
INFO:     [19:15:35] 🗂️  I will conduct my research based on the following queries: ['Reasons for Nvidia stock surge June 2025 AI demand earnings export restrictions', 'How generative AI and data-center revenue are driving Nvidia stock growth in 2025', "Role of Nvidia's market share and competitive advantages in its stock performance June 2025", 'why is Nvidia stock going up?']...
INFO:     [19:15:35] 
🔍 Running research for 'Reasons for Nvidia stock surge June 2025 AI demand earnings export restrictions'...
INFO:     [19:15:35]  Getting relevant content based on query: Reasons for Nvidia stock surge June 2025 AI demand earnings export restrictions...
INFO:     [19:15:35] 
🔍 Running research 

In [30]:
report = await researcher.write_report()  # Write the research report

INFO:     [19:15:48] ✍️ Writing report for 'why is Nvidia stock going up?'...
INFO:     [19:16:44] 📝 Report written for 'why is Nvidia stock going up?'


In [6]:
# custom_logs_handler.logs